# *Weak lensing* maps from Takahashi et al. simulations

This notebooks converts binary data and gets map information for weak lensing convergence and galactic *shear*

In [1]:
import numpy as np
import healpy as hp
import os

## Script to automatically download data files (inputs) from the url of Takahashi et al. 
http://cosmo.phys.hirosaki-u.ac.jp/takahasi/allsky_raytracing/nres12.html

In [ ]:
import urllib.request
import time
         # http://cosmo.phys.hirosaki-u.ac.jp/takahasi/allsky_raytracing/sub1/nres12/allskymap_nres12r010.zs1.mag.dat

# defining the url string as the first part of the link
string1 = 'http://cosmo.phys.hirosaki-u.ac.jp/takahasi/allsky_raytracing/sub1/nres12/allskymap_nres12r'

Select the range of realizations you want to use (depending on computer capacity)

In [2]:
start = time.time()
for i in range(10,15): # 108 realizations in total (JUST ONE z each)
    if i < 10:
        datafiles_url = string1 +'00'+ str(i)+'.zs1.mag.dat'
        local_file = 'allskymap_nres12r00'+str(i)+'.zs1.mag.dat' # downloads and save in the current directory with the name given
        urllib.request.urlretrieve(datafiles_url, local_file)
    if 10 <= i <= 99:
        datafiles_url = string1 +'0'+ str(i)+'.zs1.mag.dat'
        # print(datafiles_url)
        # print(datafiles_url == 'http://cosmo.phys.hirosaki-u.ac.jp/takahasi/allsky_raytracing/sub1/nres12/allskymap_nres12r011.zs1.mag.dat')

        local_file = 'allskymap_nres12r0'+str(i)+'.zs1.mag.dat'
        urllib.request.urlretrieve(datafiles_url, local_file)
    if 99 < i:
        datafiles_url = string1 + str(i)+'.zs1.mag.dat'
        local_file = 'allskymap_nres12r'+str(i)+'.zs1.mag.dat'
        urllib.request.urlretrieve(datafiles_url, local_file)

end = time.time()

print('Time:', end - start)

Time: 785.5836420059204


# Constructing an array from data in *binary* file

From **rxxx.zs##.mag.dat** file takes information in order to define the next arrays:

### Reading binary data

In [3]:
def make_experiment(filename, num_realization):
    # define array
    skip = [0, 536870908, 1073741818, 1610612728, 2147483638, 2684354547, 3221225457]
    load_blocks = [skip[i+1]-skip[i] for i in range(0, 6)] # array of shape (6, ) of blocks for __??__ (values)
    
    with open(filename, 'rb') as f:  # allows simplifying writing
        rec = np.fromfile(f, dtype='uint32', count=1)[0]  # define array of specific variable type 
        nside = np.fromfile(f, dtype='int32', count=1)[0] # (but now in particular just takes a single item (count=1 -> just one, count=-1 -> all))
        npix = np.fromfile(f, dtype='int64', count=1)[0]  # and takes the index [0], so it's almost like an individual float (confirmed w/ type)
        rec = np.fromfile(f, dtype='uint32', count=1)[0]
        print("nside:{} npix:{}".format(nside, npix))     # prints the corresponding N_side = 4096

        rec = np.fromfile(f, dtype='uint32', count=1)[0]  # why again?

        ############  CONVERGENCE part ############

#         kappa = np.array([])    #empty array
#         r = npix                # redefine the previous variable from f

#         for i, l in enumerate(load_blocks):  # similar to for+range but with an iterator (two args: counter, values)
#             blocks = min(l, r)  # define the min. between the **values of load_blocks** and **num. pix.**
#             load = np.fromfile(f, dtype='float32', count=blocks) # now takes n=blocks count (# of items)
#             np.fromfile(f, dtype='uint32', count=2) # takes 2 items from f
#             kappa = np.append(kappa, load) # add these values to the initial empty array
#             r = r-blocks # takes away the minimum

#             if r == 0:  # we avoid the case r=0
#                 break
#             elif r > 0 and i == len(load_blocks)-1:
#                 load = np.fromfile(f, dtype='float32', count=r)  # def. array of 'r' elements
#                 np.fromfile(f, dtype='uint32', count=2)
#                 kappa = np.append(kappa, load)

        ############  SHEAR part (1) ############

        gamma1 = np.array([])
        r = npix

        for i, l in enumerate(load_blocks):
            blocks = min(l, r)
            load = np.fromfile(f, dtype='float32', count=blocks)
            np.fromfile(f, dtype='uint32', count=2)
            gamma1 = np.append(gamma1, load)
            r = r-blocks
            if r == 0:
                break
            elif r > 0 and i == len(load_blocks)-1:
                load = np.fromfile(f, dtype='float32', count=r)
                np.fromfile(f, dtype='uint32', count=2)
                gamma1 = np.append(gamma1, load)

        ############  SHEAR part (2) ############

        gamma2 = np.array([])
        r = npix
        for i, l in enumerate(load_blocks):
            blocks = min(l, r)
            load = np.fromfile(f, dtype='float32', count=blocks)
            np.fromfile(f, dtype='uint32', count=2)
            gamma2 = np.append(gamma2, load)
            r = r-blocks
            if r == 0:
                break
            elif r > 0 and i == len(load_blocks)-1:
                load = np.fromfile(f, dtype='float32', count=r)
                np.fromfile(f, dtype='uint32', count=2)
                gamma2 = np.append(gamma2, load)

    print('loading completed')
    
    # name_kappa = 'output_k_' + str(num_realization) + '.fits'
    name_gamma1 = 'output_g1_' + str(num_realization) + '.fits'
    name_gamma2 = 'output_g2_' + str(num_realization) + '.fits'
    
    # hp.fitsfunc.write_map(name_kappa, kappa)
    hp.fitsfunc.write_map(name_gamma1, gamma1)
    hp.fitsfunc.write_map(name_gamma2, gamma2)
    
    return gamma1, gamma2, npix

# Saving data as **fits** files

**healpy.fitsfunc.write_map** (*filename, array 'm' to be written as a map)*

--> Writes a healpix map into a healpix FITS file.

In [5]:
# input file

for i in range(10,15): # *** 108 in total. Same as before: choose interval
    if i < 10:
        filename = 'allskymap_nres12r00'+str(i)+'.zs1.mag.dat'
        gamma1, gamma2, npix = make_experiment(filename, i+1)
    if 10 <= i <= 99:
        filename = 'allskymap_nres12r0'+str(i)+'.zs1.mag.dat'
        gamma1, gamma2, npix = make_experiment(filename, i+1)
    if 99 < i:
        filename = 'allskymap_nres12r'+str(i)+'.zs1.mag.dat'
        gamma1, gamma2, npix = make_experiment(filename, i+1)

nside:4096 npix:201326592
loading completed


nside:4096 npix:201326592
loading completed
nside:4096 npix:201326592
loading completed
nside:4096 npix:201326592
loading completed
nside:4096 npix:201326592
loading completed


## Printing data (individually)

This cell shows on screen the *WL* data from the contributions of **convergence and shear** (both components) as


| $\kappa$ | $\gamma_1$ | $\gamma_2$ |
| --- | --- | --- |
| - | - | - |

In [ ]:
# I recomend not to print it bc is tooo much :) unnecesary

# for i in range (npix):    # just in order to see it (e.g., r107)
#         print(i, kappa[i], gamma1[i], gamma2[i]) #, omega[i])

### Deleting initial datafiles (.mag.dat)

In order to not having n repeated copies of the same file every time the code is run (it automatically downloads every time its excecuted)

In [6]:
for j in range(10,15): # *** 108 in total again
    if i < 10:
        os.remove( 'allskymap_nres12r00'+ str(j)+'.zs1.mag.dat')
    if 10 <= i <= 99:
        os.remove('allskymap_nres12r0'+ str(j)+'.zs1.mag.dat')
    if 99 < i:
        os.remove('allskymap_nres12r' + str(j)+'.zs1.mag.dat')

It's done :) now you can go to the next notebook which computes the 2PCF of shear using these fits maps...